In [1]:
!ls

Backup_test_dataset.ipynb
alvaro_model_optimization.ipynb
alvaro_model_prototype.ipynb
gceline_LDA_topic_modelling.ipynb
gceline_LDA_topic_modelling_v2.ipynb
gceline_dataframe_preparation.ipynb
gceline_frontend_streamlit.ipynb
lda.html
pipeline.ipynb
yunxi_2Classes.ipynb
yunxi_feature_extraction.ipynb
yunxi_model_RandomForest_etc.ipynb
yunxi_preprocessing_test.ipynb
yunxis_copy_of_alvaro_model_prototype.ipynb


In [2]:
cd ..

/Users/yunxizzy/code/theresa-sporn/WorkforceSentimentMonitoring


In [1]:
%load_ext autoreload
%autoreload 2

# Prepare Data

## Load Datasets

In [2]:
# Get data with our own functions, train & test splits
from WorkforceSentimentMonitoring.data import get_data, merge, holdout

In [6]:
submission, train, test = get_data()

In [ ]:
train.head()

In [42]:
# merge corresponding dataframes
df = merge(submission, train, test)

In [43]:
df.head()

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review
0,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,NaN,4,5,5,4,5,5,Best Company to work for People are smart and ...
1,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,2,3,3,5,3,5,"Moving at the speed of light, burn out is inev..."
2,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5,4,5,5,4,5,Great balance between big-company security and...
3,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,2,5,5,4,5,4,The best place I've worked and also the most d...
4,Execellent for engineers,Impact driven. Best tech in the world.,Size matters. Engineers are a bit disconnected...,NaN,5,5,5,5,5,4,Execellent for engineers Impact driven. Best t...


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52815 entries, 0 to 29271
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   summary               52734 non-null  object
 1   positives             52815 non-null  object
 2   negatives             52814 non-null  object
 3   advice_to_mgmt        30132 non-null  object
 4   work-balance          52815 non-null  uint8 
 5   culture-values        52815 non-null  uint8 
 6   career-opportunities  52815 non-null  uint8 
 7   comp-benefits         52815 non-null  uint8 
 8   senior-mgmt           52815 non-null  uint8 
 9   overall               52815 non-null  uint8 
 10  review                52815 non-null  object
dtypes: object(5), uint8(6)
memory usage: 2.7+ MB


In [45]:
# small sample of dataframe ==> random 10_000 rows
small_df = df.sample(n=10_000, random_state=1, replace=True)

In [46]:
small_df.head()

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review
9204,Terrible Company,Decent pay and health plan.,"Stressful work environment, very high quotas, ...",Stop treating your workers like trash.,1,1,1,2,1,1,Terrible Company Decent pay and health plan. S...
14938,Amazon is great to buy and sell products in th...,its really cool place to work with amazon,itsw great job to work .join with us.,make easy to post products in facebook .,5,5,5,5,5,5,Amazon is great to buy and sell products in th...
6640,Low paymasters,Good quality projects to work on.,Unrealistic deadlines and if you have a bad bo...,Dont bring in people at the wrong pay grade/le...,2,3,3,1,3,3,Low paymasters Good quality projects to work o...
8709,Buying,"Startup Feel, Fun Environment, Great Company","Long hours, Poor Management, Growth",NaN,2,3,3,3,2,2,"Buying Startup Feel, Fun Environment, Great Co..."
26452,Ok,"Smart people, good compensation, the opportuni...",A lack of good communication between the diffe...,NaN,3,3,4,5,3,3,"Ok Smart people, good compensation, the opport..."


In [47]:
# rest index to ordinal
small_df = small_df.reset_index(drop=True)

In [9]:
small_df.head()

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review
0,Terrible Company,Decent pay and health plan.,"Stressful work environment, very high quotas, ...",Stop treating your workers like trash.,1,1,1,2,1,1,Terrible Company Decent pay and health plan. S...
1,Amazon is great to buy and sell products in th...,its really cool place to work with amazon,itsw great job to work .join with us.,make easy to post products in facebook .,5,5,5,5,5,5,Amazon is great to buy and sell products in th...
2,Low paymasters,Good quality projects to work on.,Unrealistic deadlines and if you have a bad bo...,Dont bring in people at the wrong pay grade/le...,2,3,3,1,3,3,Low paymasters Good quality projects to work o...
3,Buying,"Startup Feel, Fun Environment, Great Company","Long hours, Poor Management, Growth",NaN,2,3,3,3,2,2,"Buying Startup Feel, Fun Environment, Great Co..."
4,Ok,"Smart people, good compensation, the opportuni...",A lack of good communication between the diffe...,NaN,3,3,4,5,3,3,"Ok Smart people, good compensation, the opport..."


## Prepare Features & Targets

### Prepare Feature columns

In [48]:
# select out the text cols
text_columns = small_df.select_dtypes('object').columns
# dataframe of text columns
text_df = small_df[text_columns]
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   summary         9992 non-null   object
 1   positives       10000 non-null  object
 2   negatives       10000 non-null  object
 3   advice_to_mgmt  5709 non-null   object
 4   review          10000 non-null  object
dtypes: object(5)
memory usage: 390.8+ KB


In [94]:
# import temporary feature engineering packages
from WorkforceSentimentMonitoring.trainer import Trainer, FEATURE_COLS, SCORE_COLS
from WorkforceSentimentMonitoring.feature_engineering import get_lengths, getPolarity, getSubjectivity, get_subjectivity_polarity_columns, add_multinomial_nb_prediction_feature, get_linear_regression_cols, minmax_when_needed, train_logReg

In [95]:
from WorkforceSentimentMonitoring.preprocessing import preprocessing

In [51]:
text_df = text_df.astype('U').applymap(preprocessing)

In [52]:
# sample of preprocessed text columns
small_df[text_df.columns] = text_df
small_df.head()

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review
0,terrible company,decent pay health plan,stressful work environment high quota worker s...,stop treating worker like trash,1,1,1,2,1,1,terrible company decent pay health plan stress...
1,amazon great buy sell product world,really cool place work amazon,itsw great job work join u,make easy post product facebook,5,5,5,5,5,5,amazon great buy sell product world really coo...
2,low paymaster,good quality project work,unrealistic deadline bad bos life tough,dont bring people wrong pay grade level lookin...,2,3,3,1,3,3,low paymaster good quality project work unreal...
3,buying,startup feel fun environment great company,long hour poor management growth,nan,2,3,3,3,2,2,buying startup feel fun environment great comp...
4,ok,smart people good compensation opportunity wor...,lack good communication different business org...,nan,3,3,4,5,3,3,ok smart people good compensation opportunity ...


#### Feature Engineering #1

In [96]:
small_df = get_lengths(small_df)

100%|██████████| 5/5 [00:00<00:00, 52.29it/s]


In [97]:
small_df.head()

,summary,positives,negatives,advice_to_mgmt,review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,summary_length,postives_length,negatives_length,advice_length,combined_length,positives_length,advice_to_mgmt_length,review_length
0,terrible company,decent pay health plan,stressful work environment high quota worker s...,stop treating worker like trash,terrible company decent pay health plan stress...,1,1,1,2,1,1,16,22,60,31,132,22,31,132
1,amazon great buy sell product world,really cool place work amazon,itsw great job work join u,make easy post product facebook,amazon great buy sell product world really coo...,5,5,5,5,5,5,35,29,26,31,124,29,31,124
2,low paymaster,good quality project work,unrealistic deadline bad bos life tough,dont bring people wrong pay grade level lookin...,low paymaster good quality project work unreal...,2,3,3,1,3,3,13,25,39,71,151,25,71,151
3,buying,startup feel fun environment great company,long hour poor management growth,nan,buying startup feel fun environment great comp...,2,3,3,3,2,2,6,42,32,3,86,42,3,86
4,ok,smart people good compensation opportunity wor...,lack good communication different business org...,nan,ok smart people good compensation opportunity ...,3,3,4,5,3,3,2,76,55,3,139,76,3,139


In [56]:
small_df = small_df[['summary', 'positives', 'negatives', 'advice_to_mgmt', 'review', 'work-balance','culture-values', 'career-opportunities', 'comp-benefits', 'senior-mgmt', 'overall', 'summary_length', 'postives_length', 'negatives_length', 'advice_length', 'combined_length']]
small_df.head()

,summary,positives,negatives,advice_to_mgmt,review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,summary_length,postives_length,negatives_length,advice_length,combined_length
0,terrible company,decent pay health plan,stressful work environment high quota worker s...,stop treating worker like trash,terrible company decent pay health plan stress...,1,1,1,2,1,1,16,22,60,31,132
1,amazon great buy sell product world,really cool place work amazon,itsw great job work join u,make easy post product facebook,amazon great buy sell product world really coo...,5,5,5,5,5,5,35,29,26,31,124
2,low paymaster,good quality project work,unrealistic deadline bad bos life tough,dont bring people wrong pay grade level lookin...,low paymaster good quality project work unreal...,2,3,3,1,3,3,13,25,39,71,151
3,buying,startup feel fun environment great company,long hour poor management growth,nan,buying startup feel fun environment great comp...,2,3,3,3,2,2,6,42,32,3,86
4,ok,smart people good compensation opportunity wor...,lack good communication different business org...,nan,ok smart people good compensation opportunity ...,3,3,4,5,3,3,2,76,55,3,139


In [18]:
# print(SCORE_COLS)

In [57]:
# rename the variable
df = small_df.copy()

In [58]:
df.head()

,summary,positives,negatives,advice_to_mgmt,review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,summary_length,postives_length,negatives_length,advice_length,combined_length
0,terrible company,decent pay health plan,stressful work environment high quota worker s...,stop treating worker like trash,terrible company decent pay health plan stress...,1,1,1,2,1,1,16,22,60,31,132
1,amazon great buy sell product world,really cool place work amazon,itsw great job work join u,make easy post product facebook,amazon great buy sell product world really coo...,5,5,5,5,5,5,35,29,26,31,124
2,low paymaster,good quality project work,unrealistic deadline bad bos life tough,dont bring people wrong pay grade level lookin...,low paymaster good quality project work unreal...,2,3,3,1,3,3,13,25,39,71,151
3,buying,startup feel fun environment great company,long hour poor management growth,nan,buying startup feel fun environment great comp...,2,3,3,3,2,2,6,42,32,3,86
4,ok,smart people good compensation opportunity wor...,lack good communication different business org...,nan,ok smart people good compensation opportunity ...,3,3,4,5,3,3,2,76,55,3,139


#### Train Text split

In [59]:
small_X_train, small_X_test, small_y_train, small_y_test = holdout(df, SCORE_COLS)

In [60]:
small_X_train

,summary,positives,negatives,advice_to_mgmt,review,summary_length,postives_length,negatives_length,advice_length,combined_length
0,risk investigation,growth fast working environment good,con amazon good place work,advice,risk investigation growth fast working environ...,18,36,26,6,89
1,program manager,fast evolving opportunity learn,unorganized process work structure,nan,program manager fast evolving opportunity lear...,15,31,34,3,86
2,operation specialist,good benefit decent pay great coworkers,hard move work politics play management often ...,le micromanaging,operation specialist good benefit decent pay g...,20,39,80,16,158
3,great first long term,secure job microsoft going away anytime soon g...,review system set stone curve pitting people i...,value long term employee,great first long term secure job microsoft goi...,21,127,179,24,354
4,great place dare great,great pay excellent people high standard bette...,business unit exceed expectation pressure real...,nan,great place dare great great pay excellent peo...,22,87,98,3,213
...,...,...,...,...,...,...,...,...,...,...
6995,csa psc kaizen representative training assistant,terrific work environment excellent management...,honestly think con working amazon start sooner...,advice management psc keep giving agent opport...,csa psc kaizen representative training assista...,48,223,162,185,621
6996,genius bar,good company apple nice store,nothing bad good,good,genius bar good company apple nice store nothi...,10,29,16,4,62
6997,software engineer,work along side many smart people,secrecy make harder collaborate,nan,software engineer work along side many smart p...,17,33,31,3,87
6998,good place working,challenging good way want prove ability advance,harsh environment hard work fulfill customer o...,sometimes get brilliant idea listen worker,good place working challenging good way want p...,18,47,69,42,179


In [61]:
small_y_train

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,5,5,5,4
1,2,4,5,5,4,4
2,3,3,3,4,2,2
3,4,2,3,5,2,4
4,3,5,4,5,3,3
...,...,...,...,...,...,...
6995,5,5,5,4,5,5
6996,5,5,5,5,5,4
6997,5,5,5,5,5,5
6998,4,4,4,5,5,3


#### Feature Engineering #2

In [62]:
# vectorized and NB to generate the features
small_X_train = add_multinomial_nb_prediction_feature(small_X_train, small_y_train)
small_X_test = add_multinomial_nb_prediction_feature(small_X_test, small_y_test)

In [63]:
small_X_train

,summary,positives,negatives,advice_to_mgmt,review,summary_length,postives_length,negatives_length,advice_length,combined_length,...,summary_senior-mgmt_nb,positives_senior-mgmt_nb,negatives_senior-mgmt_nb,advice_to_mgmt_senior-mgmt_nb,review_senior-mgmt_nb,summary_overall_nb,positives_overall_nb,negatives_overall_nb,advice_to_mgmt_overall_nb,review_overall_nb
0,risk investigation,growth fast working environment good,con amazon good place work,advice,risk investigation growth fast working environ...,18,36,26,6,89,...,4,4,5,5,4,4,4,4,4,4
1,program manager,fast evolving opportunity learn,unorganized process work structure,nan,program manager fast evolving opportunity lear...,15,31,34,3,86,...,4,4,3,4,4,4,4,4,4,4
2,operation specialist,good benefit decent pay great coworkers,hard move work politics play management often ...,le micromanaging,operation specialist good benefit decent pay g...,20,39,80,16,158,...,4,3,4,4,4,4,4,4,4,4
3,great first long term,secure job microsoft going away anytime soon g...,review system set stone curve pitting people i...,value long term employee,great first long term secure job microsoft goi...,21,127,179,24,354,...,2,3,3,3,4,4,4,4,3,4
4,great place dare great,great pay excellent people high standard bette...,business unit exceed expectation pressure real...,nan,great place dare great great pay excellent peo...,22,87,98,3,213,...,3,4,4,4,4,3,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,csa psc kaizen representative training assistant,terrific work environment excellent management...,honestly think con working amazon start sooner...,advice management psc keep giving agent opport...,csa psc kaizen representative training assista...,48,223,162,185,621,...,5,5,5,5,5,5,4,4,5,4
6996,genius bar,good company apple nice store,nothing bad good,good,genius bar good company apple nice store nothi...,10,29,16,4,62,...,5,4,5,5,4,4,4,4,4,4
6997,software engineer,work along side many smart people,secrecy make harder collaborate,nan,software engineer work along side many smart p...,17,33,31,3,87,...,4,5,5,4,5,4,3,4,4,4
6998,good place working,challenging good way want prove ability advance,harsh environment hard work fulfill customer o...,sometimes get brilliant idea listen worker,good place working challenging good way want p...,18,47,69,42,179,...,4,5,5,3,5,4,3,3,4,3


#### Feature Engineering #3

In [64]:
# extract subjectivity and polarity
small_X_train = get_subjectivity_polarity_columns(small_X_train)
small_X_test = get_subjectivity_polarity_columns(small_X_test)

100%|██████████| 5/5 [00:06<00:00,  1.37s/it]


In [65]:
small_X_train

,summary,positives,negatives,advice_to_mgmt,review,summary_length,postives_length,negatives_length,advice_length,combined_length,...,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,risk investigation,growth fast working environment good,con amazon good place work,advice,risk investigation growth fast working environ...,18,36,26,6,89,...,0.000000,0.000000,0.600000,0.450000,0.600000,7.000000e-01,0.000000,0.00,0.600000,0.533333
1,program manager,fast evolving opportunity learn,unorganized process work structure,nan,program manager fast evolving opportunity lear...,15,31,34,3,86,...,0.000000,0.000000,0.600000,0.200000,0.000000,0.000000e+00,0.000000,0.00,0.600000,0.200000
2,operation specialist,good benefit decent pay great coworkers,hard move work politics play management often ...,le micromanaging,operation specialist good benefit decent pay g...,20,39,80,16,158,...,0.000000,0.000000,0.672222,0.555556,0.541667,-2.916667e-01,0.000000,0.00,0.639583,0.343750
3,great first long term,secure job microsoft going away anytime soon g...,review system set stone curve pitting people i...,value long term employee,great first long term secure job microsoft goi...,21,127,179,24,354,...,0.494444,0.333333,0.408571,0.242857,0.474667,-8.000000e-03,0.400000,-0.05,0.449966,0.151735
4,great place dare great,great pay excellent people high standard bette...,business unit exceed expectation pressure real...,nan,great place dare great great pay excellent peo...,22,87,98,3,213,...,0.750000,0.800000,0.558000,0.492000,1.000000,1.000000e+00,0.000000,0.00,0.661250,0.632500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,csa psc kaizen representative training assistant,terrific work environment excellent management...,honestly think con working amazon start sooner...,advice management psc keep giving agent opport...,csa psc kaizen representative training assista...,48,223,162,185,621,...,0.000000,0.000000,0.850000,0.570000,0.662500,4.625000e-01,0.200000,0.10,0.663636,0.445455
6996,genius bar,good company apple nice store,nothing bad good,good,genius bar good company apple nice store nothi...,10,29,16,4,62,...,0.000000,0.000000,0.800000,0.650000,0.633333,5.551115e-17,0.600000,0.70,0.693333,0.400000
6997,software engineer,work along side many smart people,secrecy make harder collaborate,nan,software engineer work along side many smart p...,17,33,31,3,87,...,0.000000,0.000000,0.571429,0.357143,0.000000,-1.000000e-01,0.000000,0.00,0.380952,0.204762
6998,good place working,challenging good way want prove ability advance,harsh environment hard work fulfill customer o...,sometimes get brilliant idea listen worker,good place working challenging good way want p...,18,47,69,42,179,...,0.600000,0.700000,0.800000,0.600000,0.620833,-2.458333e-01,1.000000,0.90,0.740278,0.384722


#### Scaling & Feature Engineering #4

In [66]:
# minmaxscale the value to be centered around (0,1)
small_X_train, small_X_test = minmax_when_needed(small_X_train, small_X_test)

In [67]:
# drop non-numerical features
small_X_train.drop(columns=FEATURE_COLS, inplace=True)
small_X_test.drop(columns=FEATURE_COLS, inplace=True)

In [68]:
small_X_train

,summary_length,postives_length,negatives_length,advice_length,combined_length,summary_work-balance_nb,positives_work-balance_nb,negatives_work-balance_nb,advice_to_mgmt_work-balance_nb,review_work-balance_nb,...,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,0.178218,0.015306,0.004226,0.003344,0.009861,1.00,0.50,1.00,1.00,1.00,...,0.000000,0.000000,0.600000,0.450000,0.600000,7.000000e-01,0.000000,0.00,0.600000,0.533333
1,0.148515,0.012987,0.005635,0.001672,0.009399,0.75,0.75,0.75,0.75,0.75,...,0.000000,0.000000,0.600000,0.200000,0.000000,0.000000e+00,0.000000,0.00,0.600000,0.200000
2,0.198020,0.016698,0.013735,0.008919,0.020493,0.75,0.50,0.50,0.50,0.50,...,0.000000,0.000000,0.672222,0.555556,0.541667,-2.916667e-01,0.000000,0.00,0.639583,0.343750
3,0.207921,0.057514,0.031167,0.013378,0.050693,0.50,0.75,0.75,0.50,0.75,...,0.494444,0.333333,0.408571,0.242857,0.474667,-8.000000e-03,0.400000,-0.05,0.449966,0.151735
4,0.217822,0.038961,0.016904,0.001672,0.028968,0.50,0.75,0.50,0.75,0.75,...,0.750000,0.800000,0.558000,0.492000,1.000000,1.000000e+00,0.000000,0.00,0.661250,0.632500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0.475248,0.102041,0.028174,0.103122,0.091834,1.00,1.00,1.00,1.00,1.00,...,0.000000,0.000000,0.850000,0.570000,0.662500,4.625000e-01,0.200000,0.10,0.663636,0.445455
6996,0.099010,0.012059,0.002465,0.002230,0.005701,0.50,1.00,1.00,0.75,1.00,...,0.000000,0.000000,0.800000,0.650000,0.633333,5.551115e-17,0.600000,0.70,0.693333,0.400000
6997,0.168317,0.013915,0.005107,0.001672,0.009553,1.00,0.75,0.50,0.75,0.75,...,0.000000,0.000000,0.571429,0.357143,0.000000,-1.000000e-01,0.000000,0.00,0.380952,0.204762
6998,0.178218,0.020408,0.011798,0.023411,0.023729,0.75,0.75,0.75,0.75,0.75,...,0.600000,0.700000,0.800000,0.600000,0.620833,-2.458333e-01,1.000000,0.90,0.740278,0.384722


In [69]:
small_X_train, small_X_test = get_linear_regression_cols(small_X_train, small_X_test, small_y_train)

NameError: name 'score_cols' is not defined

In [32]:
small_X_train

,summary_length,postives_length,negatives_length,advice_length,combined_length,summary_work-balance_nb,positives_work-balance_nb,negatives_work-balance_nb,advice_to_mgmt_work-balance_nb,review_work-balance_nb,...,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review,work-balance_regression,culture-values_regression,career-opportunities_regression,comp-benefits_regression,senior-mgmt_regression,overall_regression
0,0.128713,0.022452,0.006865,0.001609,0.013251,1.00,1.00,1.00,0.75,1.00,...,0.000000,0.000000,0.661869,0.106061,4.908659,4.929658,4.275579,4.922897,4.588032,4.226959
1,0.138614,0.027840,0.008977,0.009115,0.019260,0.75,0.75,0.75,0.75,0.75,...,0.000000,0.000000,0.000000,0.000000,4.001545,4.296191,4.221158,3.884448,4.932942,4.547852
2,0.029703,0.062865,0.025348,0.088472,0.066718,0.75,0.75,0.75,0.50,0.75,...,0.616667,0.050000,0.545000,0.125000,3.122168,3.342215,2.797256,3.741536,2.625563,3.289185
3,0.396040,0.255052,0.071466,0.184450,0.206317,0.75,0.75,0.75,0.75,0.75,...,0.627083,0.141004,0.506548,0.145801,3.703233,3.683149,3.998842,3.469999,2.652609,2.662354
4,0.118812,0.026493,0.007041,0.045040,0.027119,0.50,0.50,0.50,0.50,0.50,...,0.000000,0.000000,0.791667,0.247222,1.987921,2.982082,2.514659,3.332644,2.289998,2.720184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0.158416,0.017512,0.003521,0.001609,0.009091,1.00,1.00,0.75,0.75,1.00,...,0.000000,0.000000,0.708333,0.050000,4.296810,4.685642,4.564432,4.615076,4.275294,3.996674
6996,0.178218,0.023350,0.016370,0.043968,0.034823,0.50,0.75,0.75,0.50,0.50,...,0.500000,-0.200000,0.631548,0.031548,2.879164,3.404100,3.360514,3.622128,2.867048,2.999054
6997,0.168317,0.115851,0.082204,0.730831,0.321418,0.75,0.75,0.75,0.75,0.75,...,0.430247,0.062584,0.526733,0.096263,4.003470,3.118623,3.414131,4.643253,2.659120,3.139343
6998,0.277228,0.035474,0.017074,0.001609,0.028968,0.50,0.50,0.75,0.75,0.50,...,0.000000,0.000000,0.611224,0.402041,3.120543,3.707178,3.497243,4.368154,3.285279,3.145630


### Prepare Target columns

In [34]:
small_y_train

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,3,5,5,4
1,4,4,4,4,5,4
2,4,3,2,2,2,3
3,4,5,5,4,3,3
4,3,1,4,2,3,4
...,...,...,...,...,...,...
6995,5,5,5,5,5,5
6996,3,4,4,4,3,3
6997,4,1,2,3,2,2
6998,3,4,3,5,3,3


# Train Model

## LogReg with default params

In [91]:
trainer_rf_v2 = Trainer(small_X_train, small_X_test, estimator='Logic')

In [92]:
trainer_rf_v2.train(small_y_train, small_y_test)

  0%|          | 0/6 [00:00<?, ?it/s]

LogisticRegression


 17%|█▋        | 1/6 [00:02<00:12,  2.47s/it]

LogisticRegression


 33%|███▎      | 2/6 [00:04<00:09,  2.44s/it]

...

LogisticRegression


 83%|████████▎ | 5/6 [00:11<00:02,  2.29s/it]

LogisticRegression


100%|██████████| 6/6 [00:14<00:00,  2.38s/it]


(LogisticRegression(max_iter=10000),
 {'work-balance': 0.659,
  'culture-values': 0.62,
  'career-opportunities': 0.6486666666666666,
  'comp-benefits': 0.6816666666666666,
  'senior-mgmt': 0.6806666666666666,
  'overall': 0.6383333333333333})

## RandomForest

### RandomForest with default params

In [50]:
train_rf = Trainer(small_X_train, small_X_test)

In [52]:
predictions, pred_scores = train_rf.train(small_y_train, small_y_test)

RandomForestClassifier
RandomForestClassifier
RandomForestClassifier
RandomForestClassifier
RandomForestClassifier
RandomForestClassifier


In [53]:
predictions

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,5,5,5,5
1,4,4,5,5,5,5
2,4,5,5,5,5,5
3,5,5,5,5,5,5
4,5,5,5,5,5,5
...,...,...,...,...,...,...
2995,1,1,1,5,1,3
2996,4,3,3,4,3,4
2997,3,4,5,4,3,4
2998,3,4,5,4,4,4


In [54]:
pred_scores

{'work-balance': 0.7716666666666666,
 'culture-values': 0.6883333333333334,
 'career-opportunities': 0.7546666666666667,
 'comp-benefits': 0.737,
 'senior-mgmt': 0.7913333333333333,
 'overall': 0.6976666666666667}

### RandomForest with selected params

In [106]:
trainer_rf_v1 = Trainer(small_X_train, small_X_test, estimator='RandomForest')

In [107]:
predictions, pred_scores = trainer_rf_v1.train(small_y_train, small_y_test)

RandomForestClassifier
              precision    recall  f1-score   support

           1       0.60      0.75      0.67       298
           2       0.39      0.51      0.44       316
           3       0.77      0.71      0.74       781
           4       0.81      0.72      0.76       898
           5       0.84      0.83      0.83       707

    accuracy                           0.72      3000
   macro avg       0.68      0.70      0.69      3000
weighted avg       0.74      0.72      0.73      3000

RandomForestClassifier
              precision    recall  f1-score   support

           1       0.60      0.54      0.57       314
           2       0.35      0.22      0.27       446
           3       0.44      0.37      0.40       600
           4       0.49      0.66      0.57       579
           5       0.73      0.80      0.77      1061

    accuracy                           0.57      3000
   macro avg       0.52      0.52      0.51      3000
weighted avg       0.56      0.

In [108]:
predictions

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,5,5,5,5
1,4,4,5,5,5,5
2,4,5,5,5,5,5
3,5,5,5,5,5,5
4,5,5,5,5,5,5
...,...,...,...,...,...,...
2995,1,1,1,1,1,1
2996,2,2,3,4,3,2
2997,3,3,5,4,3,4
2998,3,3,5,4,4,4


In [109]:
pred_scores

{'work-balance': 0.724,
 'culture-values': 0.575,
 'career-opportunities': 0.7093333333333334,
 'comp-benefits': 0.652,
 'senior-mgmt': 0.7333333333333333,
 'overall': 0.6343333333333333}

### Add GridSearch for RandomForest

In [ ]:
# params to tune: max_leaf

In [85]:
# 1). manually selected params
trainer_rf_v2 = Trainer(small_X_train, small_X_test, estimator='RandomForest')

In [86]:
trainer_rf_v2.train(small_y_train, small_y_test)

  0%|          | 0/6 [00:00<?, ?it/s]

RandomForestClassifier


 17%|█▋        | 1/6 [00:00<00:02,  1.83it/s]

RandomForestClassifier


 33%|███▎      | 2/6 [00:01<00:02,  1.84it/s]

...

RandomForestClassifier


 83%|████████▎ | 5/6 [00:02<00:00,  1.91it/s]

RandomForestClassifier


100%|██████████| 6/6 [00:03<00:00,  1.91it/s]


(RandomForestClassifier(class_weight='balanced', max_depth=3),
 {'work-balance': 0.6816666666666666,
  'culture-values': 0.5346666666666666,
  'career-opportunities': 0.6373333333333333,
  'comp-benefits': 0.5713333333333334,
  'senior-mgmt': 0.72,
  'overall': 0.619})

## AdaBoost

### AdaBoost with default params

In [79]:
trainer_ada_df = Trainer(small_X_train, small_X_test, estimator='AdaBoost')

In [80]:
predictions, pred_scores = trainer_ada_df.train(small_y_train, small_y_test)

AdaBoostClassifier
              precision    recall  f1-score   support

           1       0.57      0.64      0.60       335
           2       0.36      0.63      0.46       231
           3       0.80      0.73      0.77       791
           4       0.84      0.76      0.80       888
           5       0.87      0.80      0.83       755

    accuracy                           0.74      3000
   macro avg       0.69      0.71      0.69      3000
weighted avg       0.77      0.74      0.75      3000

AdaBoostClassifier
              precision    recall  f1-score   support

           1       0.51      0.47      0.49       306
           2       0.12      0.30      0.17       110
           3       0.47      0.59      0.52       399
           4       0.71      0.73      0.72       753
           5       0.89      0.73      0.80      1432

    accuracy                           0.67      3000
   macro avg       0.54      0.56      0.54      3000
weighted avg       0.72      0.67      

In [82]:
predictions

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,5,5,5,5
1,4,4,5,5,5,5
2,4,5,5,5,5,5
3,5,5,5,5,5,4
4,5,5,5,5,5,5
...,...,...,...,...,...,...
2995,1,1,2,5,1,1
2996,2,3,3,4,3,4
2997,3,4,5,4,3,4
2998,3,3,5,4,4,4


In [83]:
pred_scores

{'work-balance': 0.7403333333333333,
 'culture-values': 0.668,
 'career-opportunities': 0.7383333333333333,
 'comp-benefits': 0.731,
 'senior-mgmt': 0.726,
 'overall': 0.6966666666666667}

### AdaBoost with selected params

In [68]:
trainer_ada = Trainer(small_X_train, small_X_test, estimator='AdaBoost')

In [69]:
predictions, pred_scores = trainer_ada.train(small_y_train, small_y_test)

AdaBoostClassifier
AdaBoostClassifier
AdaBoostClassifier
AdaBoostClassifier
AdaBoostClassifier
AdaBoostClassifier


In [70]:
predictions

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,5,5,5,5
1,4,4,5,5,5,5
2,4,5,5,5,5,5
3,5,5,5,5,5,5
4,5,5,5,5,5,5
...,...,...,...,...,...,...
2995,1,1,1,1,1,1
2996,2,3,3,4,3,4
2997,3,4,5,4,3,4
2998,3,3,5,4,4,4


In [71]:
pred_scores

{'work-balance': 0.77,
 'culture-values': 0.671,
 'career-opportunities': 0.7646666666666667,
 'comp-benefits': 0.738,
 'senior-mgmt': 0.8046666666666666,
 'overall': 0.6886666666666666}

## XGB

### XGBClassifier with default params

In [72]:
trainer_xgb = Trainer(small_X_train, small_X_test, estimator='XGB')

In [73]:
predictions, pred_scores = trainer_xgb.train(small_y_train, small_y_test)

XGBClassifier
XGBClassifier
XGBClassifier
XGBClassifier
XGBClassifier
XGBClassifier


In [74]:
predictions

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,5,5,5,3
1,4,4,5,5,5,5
2,4,5,5,5,5,4
3,5,5,5,5,5,4
4,5,5,5,5,5,4
...,...,...,...,...,...,...
2995,1,1,1,5,1,3
2996,3,3,3,4,3,4
2997,3,4,5,4,3,4
2998,3,4,5,4,4,4


In [75]:
pred_scores

{'work-balance': 0.7866666666666666,
 'culture-values': 0.7036666666666667,
 'career-opportunities': 0.773,
 'comp-benefits': 0.7433333333333333,
 'senior-mgmt': 0.808,
 'overall': 0.7266666666666667}

## GradientBoost

### GradientBoost with default params

In [85]:
trainer_gb = Trainer(small_X_train, small_X_test, estimator='GradientBoost')

In [86]:
predictions, pred_scores = trainer_gb.train(small_y_train, small_y_test)

GradientBoostingClassifier
              precision    recall  f1-score   support

           1       0.62      0.90      0.74       258
           2       0.44      0.84      0.57       211
           3       0.89      0.72      0.79       890
           4       0.88      0.77      0.82       908
           5       0.87      0.82      0.84       733

    accuracy                           0.79      3000
   macro avg       0.74      0.81      0.75      3000
weighted avg       0.82      0.79      0.79      3000

GradientBoostingClassifier
              precision    recall  f1-score   support

           1       0.56      0.71      0.63       225
           2       0.28      0.46      0.35       170
           3       0.56      0.63      0.59       444
           4       0.72      0.73      0.72       771
           5       0.89      0.75      0.81      1390

    accuracy                           0.71      3000
   macro avg       0.60      0.65      0.62      3000
weighted avg       0.74

In [87]:
predictions

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,5,5,5,5
1,4,4,5,5,5,5
2,4,5,5,5,5,5
3,5,5,5,5,5,4
4,5,5,5,5,5,4
...,...,...,...,...,...,...
2995,1,1,1,5,1,3
2996,3,3,3,4,3,4
2997,3,4,5,4,3,4
2998,3,3,5,4,4,4


In [88]:
pred_scores

{'work-balance': 0.785,
 'culture-values': 0.7056666666666667,
 'career-opportunities': 0.775,
 'comp-benefits': 0.7476666666666667,
 'senior-mgmt': 0.8156666666666667,
 'overall': 0.7276666666666667}

## Alternative: change 5 classes into 2 classes

In [93]:
from sklearn.ensemble import RandomForestClassifier

### Prepare 2 Classed target

In [5]:
small_y_train

NameError: name 'small_y_train' is not defined